# Data Prep League of Legends Data


### Dataset overview: League of Legends Competitive Matches

Data overview:
League of Legends competitive matches between 2017-2019. The matches include the NALCS, EULCS, LCK, LMS, and CBLoL leagues as well as the World Championship and Mid-Season Invitational tournaments.


[Data source](https://oracleselixir.com/match-data/)

All data has been aggregated and released by Tim “Magic” Sevenhuysen of OraclesElixir.com.

## Importing

### Import libraries

In [0]:
# follow the result prompt from this code
# install PyDrive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# import packages
import warnings
import numpy as np
import pandas as pd
import scipy
import scipy.stats as st
import statsmodels as sm
import matplotlib.pyplot as plt
import time
import seaborn as sns

### Import raw data

In [0]:
# data importing

#2016 data
link = 'https://drive.google.com/open?id=1AWbV0qgQCznU5X_dRTN_LD0cHtCeItuk'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2016_complete.xlsx')  
lol_2016_df = pd.read_excel('2016_complete.xlsx')

#2017 
link = 'https://drive.google.com/open?id=1-THh3nRsWOxKAhrvnSijQ74wRsQUZvht'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2017_complete.xlsx')  
lol_2017_df = pd.read_excel('2017_complete.xlsx')

#2018 spring
link = 'https://drive.google.com/open?id=1zqpafe59v_nunVD03NdeMbxnGckawmwg'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2018_spring.xlsx')  
lol_2018S_df = pd.read_excel('2018_spring.xlsx')

#2018 summer
link = 'https://drive.google.com/open?id=1_SOHgDR_jumNyfLQMeS9pHxQjgD10C9u'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2018_summer.xlsx')  
lol_2018SU_df = pd.read_excel('2018_summer.xlsx')

#2018 worlds
link = 'https://drive.google.com/open?id=1eyNXIu3KF8AaMk6D1hRsKUDkHwbAVVzq'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2018_worlds.xlsx')  
lol_2018W_df = pd.read_excel('2018_worlds.xlsx')

#2019 spring
link = 'https://drive.google.com/open?id=1RDSktMhpxCDUNz1CKYOvwGADICYT3vt4'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2019_spring.xlsx')  
lol_2019S_df = pd.read_excel('2019_spring.xlsx')

#2019 summer
link = 'https://drive.google.com/open?id=1G9L2NFtKbTnYhghXsTlOk9YArWWyy8ky'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2019_summer.xlsx')  
lol_2019SU_df = pd.read_excel('2019_summer.xlsx')

#2019 worlds
link = 'https://drive.google.com/open?id=1grsJONn7I8BJmtg-BxQWLW5imgJCo3AS'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('2019_worlds.xlsx')  
lol_2019W_df = pd.read_excel('2019_worlds.xlsx')


## Overview of raw data

In [0]:
# check raw data shapes
a = lol_2017_df.shape[0]
b = lol_2018S_df.shape[0]
c = lol_2018SU_df.shape[0]
d = lol_2018W_df.shape[0]
e = lol_2019S_df.shape[0]
f = lol_2019SU_df.shape[0]
g = lol_2019W_df.shape[0]
lst=[a,b,c,d,e,f,g]
print(sum(lst)*(2/12)) #num rows for prediction
print(sum(lst)*(10/12)) #num rows for clustering


16304.0
81520.0


In [0]:
# checking missing features in lol_2016
for i in lol_2017_df.columns:
  if i not in lol_2016_df:
    print (i)

ban4
ban5
fbassist
elementals
oppelementals
firedrakes
waterdrakes
earthdrakes
airdrakes
elders
oppelders
firstmidouter
csat15
oppcsat15
csdat15


Since 2016 has less columns than 2017-2019, we're not going to use it as proven above we have enough data for both clustering and prediction



In [0]:
complete_df = pd.concat([lol_2017_df,lol_2018S_df,lol_2018SU_df,lol_2018W_df,lol_2019S_df,lol_2019SU_df,lol_2019W_df],axis=0)
complete_df.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbassist,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,...,fbarontime,teambaronkills,oppbaronkills,dmgtochamps,dmgtochampsperminute,dmgshare,earnedgoldshare,wards,wpm,wardshare,wardkills,wcpm,visionwards,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
0,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,1,Blue,Top,Lourlo,Team Liquid,Gnar,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0.0,2.0,0.0,3.0,18.0,0,0,0,0,0,0.0,0,6.73213,0,0.118188,0.827315,0,11.9418,0.0,...,21.536467,0.0,1.0,8725.0,343.729481,0.269631,0.239989,14.0,0.551543,0.184211,2.0,0.078792,5,5.0,,,9067.0,224.749048,9025.0,0.028090,231.0,4.0,3.0,0.0,9.258043,3378.0,3264.0,114.0,5542.0,5419.0,123.0,4669.0,4677.0,-8.0,94.0,89.0,5.0,150.0,136.0,14.0
1,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,2,Blue,Jungle,Dardoch,Team Liquid,Gragas,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0.0,5.0,1.0,3.0,18.0,0,0,0,0,0,1.0,0,6.73213,0,0.0787919,0.827315,0,11.9418,0.0,...,21.536467,0.0,1.0,4300.0,169.402495,0.132884,0.176005,15.0,0.590939,0.197368,3.0,0.118188,5,6.0,,,7546.0,164.827840,6650.0,-0.241322,15.0,98.0,58.0,5.0,4.451740,2936.0,2883.0,53.0,4467.0,5223.0,-756.0,3134.0,3161.0,-27.0,47.0,47.0,0.0,62.0,79.0,-17.0
2,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,3,Blue,Middle,Mickey,Team Liquid,Ekko,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0.0,3.0,0.0,3.0,18.0,0,0,0,0,0,0.0,0,6.73213,0,0.315167,0.827315,0,11.9418,0.0,...,21.536467,0.0,1.0,6698.0,263.873933,0.20699,0.257531,16.0,0.630335,0.210526,4.0,0.157584,6,6.0,,,9484.0,241.177150,8700.0,-0.171789,247.0,2.0,1.0,0.0,9.809586,3446.0,3468.0,-22.0,5236.0,6286.0,-1050.0,5088.0,5169.0,-81.0,93.0,92.0,1.0,142.0,143.0,-1.0
3,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,4,Blue,ADC,Piglet,Team Liquid,Vayne,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0.0,3.0,1.0,3.0,18.0,0,0,0,0,0,1.0,0,6.73213,0,0.157584,0.827315,0,11.9418,0.0,...,21.536467,0.0,1.0,10367.0,408.417597,0.320375,0.197459,10.0,0.393959,0.131579,1.0,0.039396,2,3.0,,,8056.0,184.919764,8275.0,-0.191257,186.0,1.0,1.0,0.0,7.367039,2933.0,3287.0,-354.0,4578.0,5635.0,-1057.0,3141.0,2984.0,157.0,68.0,75.0,-7.0,112.0,128.0,-16.0
4,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,5,Blue,Support,Matt,Team Liquid,Lulu,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,3.0,5.0,0.0,3.0,18.0,1,0,0,0,1,0.0,0,6.73213,0.118188,0.0393959,0.827315,0,11.9418,0.0,...,21.536467,0.0,1.0,2269.0,89.389363,0.0701196,0.129016,21.0,0.827315,0.276316,8.0,0.315167,6,7.0,,,6429.0,120.822587,6880.0,-0.092862,12.0,0.0,0.0,0.0,0.472751,2256.0,2389.0,-133.0,3184.0,4298.0,-1114.0,2593.0,2816.0,-223.0,5.0,2.0,3.0,7.0,2.0,5.0


## Teams and Players Data Split

- Teams data will be used to build prediction model
- Players data will be used to build strategy clustering model

In [0]:
teams_df = complete_df[(complete_df['playerid'] == 100) | (complete_df['playerid'] == 200)] #only take out the team info
players_df = complete_df[complete_df['playerid']<100] #keep the players info

In [0]:
print(teams_df.columns)
teams_df.shape

Index(['gameid', 'url', 'league', 'split', 'date', 'week', 'game', 'patchno',
       'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1',
       'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'k', 'd', 'a',
       'teamkills', 'teamdeaths', 'doubles', 'triples', 'quadras', 'pentas',
       'fb', 'fbassist', 'fbvictim', 'fbtime', 'kpm', 'okpm', 'ckpm', 'fd',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elementals',
       'oppelementals', 'firedrakes', 'waterdrakes', 'earthdrakes',
       'airdrakes', 'elders', 'oppelders', 'herald', 'heraldtime', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochamps', 'dmgtochampsperminute', 'dmgshare',
       'earnedgoldshare', 'wards', 'wpm', 'wardshare', 'wardkills', 'wcpm',
       'visionwards', 'visionwardbuys', 'visiblewardclearrate',
       'invisiblewardclearrate', 'totalgold', 'earnedg

(16304, 98)

In [0]:
# subset of teams_df columns based on specific subject knowledge 
cols_teams = ['gameid','league','ban1','ban2','ban3','ban4','ban5','result','kpm','d','a','fb','fbvictim','fbtime','fd',
              'fdtime','teamdragkills','elementals','firedrakes','waterdrakes','earthdrakes','airdrakes','elders','herald','heraldtime',
              'ft','fttime','firstmidouter','firsttothreetowers','teamtowerkills','fbaron','fbarontime','teambaronkills','dmgtochamps',
              'wards','wardshare','wardkills','wcpm','visionwards','visiblewardclearrate','invisiblewardclearrate',
              'goldspent','minionkills','monsterkillsenemyjungle','goldat10','goldat15','xpat10']
print(len(cols_teams))
teams_df = teams_df[cols_teams]
print(teams_df.shape)

47
(16304, 47)


In [0]:
# checking if we have equals number of wins and losing
# make sure data integrity is still intact by checking if win-loss ratio is 1:1
print(teams_df[teams_df['result'] == 1].shape)
print(teams_df[teams_df['result'] == 0].shape)

(8152, 47)
(8152, 47)


In [0]:
#subset of players_df columns based on specific subject knowledge 

cols_players = ['gameid', 'league', 'playerid','position','champion','result','k','d','a','doubles','triples','quadras','pentas','fb','fbassist','fbtime','dmgtochamps','wards',
                'wardkills','visionwards','visionwardbuys','goldspent','minionkills','monsterkills','monsterkillsownjungle',
                'goldat10','goldat15','xpat10']
print(len(cols_players))
players_df = players_df[cols_players]
print(players_df.shape)


28
(81520, 28)


## Teams Data

### Handling missing values

In [0]:
# function to do missing value computation
def cal_missing_val(df):
    data_dict = {}
    for col in df.columns:
        data_dict[col] = df[col].isnull().sum()/df.shape[0]*100
    return pd.DataFrame.from_dict(data_dict, orient='index', columns=['MissingValue'])

In [0]:
cal_missing_val(teams_df)

,MissingValue
gameid,0.012267
league,0.000000
ban1,0.165604
ban2,0.079735
ban3,0.202404
ban4,0.196271
ban5,0.472277
result,0.000000
kpm,0.024534
d,0.024534


In [0]:
# reindex the teams data frame because it wasn't ordered after filter
# Note: reindex df after drop/split
teams_df.reset_index(drop=True, inplace=True)
teams_df.head()

,gameid,league,ban1,ban2,ban3,ban4,ban5,result,kpm,d,a,fb,fbvictim,fbtime,fd,fdtime,teamdragkills,elementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,fbaron,fbarontime,teambaronkills,dmgtochamps,wards,wardshare,wardkills,wcpm,visionwards,visiblewardclearrate,invisiblewardclearrate,goldspent,minionkills,monsterkillsenemyjungle,goldat10,goldat15,xpat10
0,1002300127,NALCS,LeBlanc,Zac,Shen,Tristana,Kog'Maw,0,0.118188,18.0,2.0,1,0,6.73213,0,11.94180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0885,1.0,12.368267,0.0,0,4.0,0.0,21.536467,0.0,32359.0,76.0,1.0,18.0,0.709127,24,0.571429,0.037037,39530.0,691.0,5.0,14949.0,23007.0,18625.0
1,1002300127,NALCS,Caitlyn,Kalista,Thresh,Blitzcrank,Bard,1,0.709127,3.0,55.0,0,1,6.73213,1,11.94180,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,14.0885,0.0,12.368267,1.0,1,10.0,1.0,21.536467,1.0,54497.0,82.0,1.0,27.0,1.063690,25,0.653846,0.2,45160.0,693.0,18.0,15291.0,26861.0,18807.0
2,240067,LCK,Xayah,Zac,Lee Sin,Nidalee,Rengar,0,0.273292,12.0,23.0,1,0,7.58223,1,11.16990,4.0,3.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,0.0,15.145150,1.0,0,3.0,0.0,24.772017,0.0,71585.0,125.0,1.0,84.0,2.086957,33,0.739726,0.306122,61970.0,1141.0,NaN,15532.0,23350.0,18849.0
3,240067,LCK,Galio,Syndra,Elise,Kennen,Fiora,1,0.298137,11.0,33.0,0,1,7.58223,0,11.16990,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,1.0,15.145150,0.0,1,11.0,1.0,24.772017,2.0,76953.0,171.0,1.0,42.0,1.043478,52,0.648649,0.204545,70475.0,1112.0,NaN,14772.0,22363.0,18788.0
4,240080,LCK,Syndra,Galio,Ashe,Kennen,Graves,0,0.168729,16.0,7.0,1,0,4.88285,0,13.74805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,12.427833,0.0,0,2.0,0.0,25.077167,0.0,50632.0,113.0,1.0,25.0,0.843645,27,0.578947,0.189189,43245.0,762.0,NaN,15131.0,24167.0,17618.0


#### Handling missing values for 'herald' variable

In [0]:
# Clarification/Cleaning 

# Replace the missing value in 'herald' column with '-1' if the gameid are the same and they are both missing
# If two rows have the same gameid and one row is 1, replace another missing value with 0
# If two rows have the same gameid and one row is 0, replace another missing value with 1
a = teams_df.isnull()['herald']

for i in range(0, len(a), 2):
  if (a[i]) & (a[i+1]):
    teams_df.loc[i, 'herald'] = -1
    teams_df.loc[i+1, 'herald'] = -1
  elif (a[i]) & (not a[i+1]):
    teams_df.loc[i, 'herald'] = 1- df.loc[i+1, 'herald']
  elif (not a[i]) & (a[i+1]):
    df.loc[i+1, 'herald'] = 1- df.loc[i, 'herald']

# If value of 'herald' equals to '-1' and 'heraldtime' is missing, make 'heraldtime' as 0
# If value of 'herald' equals to 1 or 0, check if the 'heraldtime' is missing or not, if missing, delete the rows

b = teams_df['heraldtime'].isnull()
drop_idx = []
for i in range(len(b)):
  if b[i]: 
    if (teams_df.loc[i, 'herald'] == -1):
      teams_df.loc[i, 'heraldtime'] = 0
    else:
     drop_idx.append(i)

teams_df.drop(drop_idx, inplace = True)

# reindex 
teams_df.reset_index(drop=True, inplace=True)
teams_df['herald']

0        1.0
1        0.0
2       -1.0
3       -1.0
4       -1.0
        ... 
15753    1.0
15754    0.0
15755    1.0
15756    1.0
15757    0.0
Name: herald, Length: 15758, dtype: float64

#### Removing rows with missing values

In [0]:
# Drop rows with any NaN in the selected columns only
teams_df = teams_df.dropna(how='any', subset=['ban1', 'ban2','ban3','ban4','ban5','d','a','kpm'])

# Drop rows of columns with the same amount of percentages of the missing values
teams_df = teams_df.dropna(how='all', subset=['elementals', 'firedrakes', 'waterdrakes', 'earthdrakes', 'airdrakes', 'elders', 'ft', 'firstmidouter',
                                               'firsttothreetowers', 'teambaronkills', 'dmgtochamps', 'wardshare', 'wcpm', 
                                               'minionkills',
                                               'goldat10', 'goldat15'] )

# Drop rows with missing values under the column "teamdragkills" will also drop the missing values under the columns of related 'drakes' 
# bcs they share the same percentage of missing values
teams_df = teams_df.dropna(how = 'any', subset = ['teamdragkills'])

# Drop rows with missing values under the column "minionkills" will also drop the missing values under the columns of 'monsterkills' 
# bcs they share the same percentage of missing values
teams_df = teams_df.dropna(how = 'any', subset = ['minionkills'])

# Drop rows with missing values under the column "wards" will also drop the missing values under the columns of 'visionwardbuys' 
# bcs they share the same percentage of missing values
teams_df = teams_df.dropna(how = 'any', subset = ['wards'])

# Drop rows with missing values under the column "fbaron" will also drop the missing values under the columns of 'fbarontime' 
# bcs they share the same percentage of missing values
teams_df = teams_df.dropna(how = 'any', subset = ['fbaron'])

# Drop the missing values from column 'fdtime' and 'wardkills'
teams_df = teams_df.dropna(how = 'any', subset = ['fdtime', 'wardkills'])

# Drop the missing values from column 'visionwards'
teams_df = teams_df.dropna(how = 'any', subset = ['visionwards'])

# Drop the missing values from column 'visiblewardclearrate' and 'invisiblewardclearrate'
teams_df = teams_df.dropna(how = 'any', subset = ['visiblewardclearrate', 'invisiblewardclearrate'])
teams_df = teams_df.dropna(how = 'any',subset = ['monsterkillsenemyjungle'])


# kill all the games that only get one line of info
split = teams_df.groupby('gameid').count().sort_values(by = 'league',ascending = False)
index = list(range(len(split['league'])))
split['index']= index
split = split.reset_index().set_index('index')

delete_gameid = split[split.apply(lambda x: x['league'] == 1, axis=1)]['gameid']
new_df = teams_df[~teams_df.gameid.isin(delete_gameid)]

teams_df = new_df
#reset index
teams_df.reset_index(drop = False)



,index,gameid,league,ban1,ban2,ban3,ban4,ban5,result,kpm,d,a,fb,fbvictim,fbtime,fd,fdtime,teamdragkills,elementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,fbaron,fbarontime,teambaronkills,dmgtochamps,wards,wardshare,wardkills,wcpm,visionwards,visiblewardclearrate,invisiblewardclearrate,goldspent,minionkills,monsterkillsenemyjungle,goldat10,goldat15,xpat10
0,0,1002300127,NALCS,LeBlanc,Zac,Shen,Tristana,Kog'Maw,0,0.118188,18.0,2.0,1,0,6.73213,0,11.941800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.088500,1.0,12.368267,0.0,0,4.0,0.0,21.536467,0.0,32359.0,76.0,1.0,18.0,0.709127,24,0.571429,0.037037,39530.0,691.0,5.0,14949.0,23007.0,18625.0
1,1,1002300127,NALCS,Caitlyn,Kalista,Thresh,Blitzcrank,Bard,1,0.709127,3.0,55.0,0,1,6.73213,1,11.941800,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,14.088500,0.0,12.368267,1.0,1,10.0,1.0,21.536467,1.0,54497.0,82.0,1.0,27.0,1.063690,25,0.653846,0.2,45160.0,693.0,18.0,15291.0,26861.0,18807.0
2,18,240145,NACS,Elise,Caitlyn,Thresh,Vladimir,Brand,1,0.554734,28.0,64.0,1,0,6.53215,0,16.393367,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.826250,0.0,14.792250,1.0,0,10.0,0.0,23.230017,2.0,142669.0,134.0,1.0,36.0,0.798817,23,0.538462,0.147059,80035.0,1117.0,82.0,14972.0,23524.0,17739.0
3,19,240145,NACS,Zac,Cassiopeia,Nunu,Tristana,Viktor,0,0.621302,25.0,72.0,0,1,6.53215,1,16.393367,3.0,3.0,1.0,0.0,0.0,2.0,0.0,1.0,13.826250,1.0,14.792250,0.0,1,6.0,1.0,23.230017,1.0,139501.0,141.0,1.0,52.0,1.153846,33,0.658537,0.268817,77850.0,1080.0,82.0,15051.0,24807.0,18289.0
4,24,240148,NACS,Cassiopeia,Sejuani,Blitzcrank,Lucian,Viktor,1,0.784314,2.0,35.0,1,0,5.72888,1,12.597450,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,16.359083,1.0,7.878567,1.0,1,10.0,1.0,20.965183,1.0,50721.0,68.0,1.0,14.0,0.610022,11,0.733333,0.0714286,42430.0,665.0,33.0,17010.0,27395.0,19356.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,15753,1072511,WC,Pantheon,Qiyana,Xayah,Blitzcrank,Braum,0,0.171149,22.0,13.0,0,1,3.1899,0,6.770567,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,14.422167,1.0,14.850650,1.0,1,6.0,0.0,31.127867,0.0,59489.0,199.0,1.0,93.0,2.273839,54,0.518987,0.420455,66900.0,1116.0,35.0,14864.0,23411.0,17319.0
10882,15754,1072516,WC,Renekton,Nautilus,Qiyana,Gangplank,Kayle,0,0.156047,20.0,8.0,0,1,7.4919,1,7.453850,2.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,12.709867,0.0,13.383217,0.0,0,1.0,0.0,20.264417,0.0,34076.0,101.0,1.0,39.0,1.521456,31,0.473684,0.296296,35300.0,613.0,2.0,14986.0,22110.0,17398.0
10883,15755,1072516,WC,Pantheon,Syndra,Xayah,Olaf,Rek'Sai,1,0.780234,4.0,41.0,1,0,7.4919,0,7.453850,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,12.709867,1.0,13.383217,1.0,1,9.0,1.0,20.264417,1.0,42278.0,105.0,1.0,33.0,1.287386,30,0.424242,0.3125,46520.0,676.0,14.0,16087.0,26509.0,19183.0
10884,15756,1072517,WC,Syndra,Gragas,Kai'Sa,Olaf,Rek'Sai,1,0.329308,8.0,27.0,0,1,14.2987,1,6.686750,3.0,3.0,0.0,1.0,2.0,0.0,0.0,1.0,10.591683,1.0,7.764867,0.0,1,7.0,1.0,23.078600,2.0,46751.0,142.0,1.0,64.0,2.107574,46,0.592593,0.298246,49053.0,713.0,24.0,15704.0,24618.0,17393.0


In [0]:
# check if there still exist any missing value, balance of data and shape

cal_missing_val(teams_df)
teams_df.groupby('result').count()
teams_df.shape

# shape is still okay after dropping so this method is acceptable

(10886, 47)

### Feature Engineering

#### Generate new features using goldat10 and goldat15

In [0]:
# to calculate golddiff at 10, golddiff at 15
teams_df2=teams_df
num = len(teams_df2['goldat10'])
teams_df2['gdiffat10']=""
teams_df2['gdiffat15']=""
teams_df2.reset_index(inplace=True)
# print(teams_df2['goldat10'][0+1]-teams_df2['goldat10'][0])
for i in range(num):
  if (i%2==0):
    teams_df2['gdiffat10'][i] = teams_df2['goldat10'][i]-teams_df2['goldat10'][i+1]
    teams_df2['gdiffat15'][i] = teams_df2['goldat15'][i]-teams_df2['goldat15'][i+1]
  else:
    teams_df2['gdiffat10'][i] = teams_df2['goldat10'][i]-teams_df2['goldat10'][i-1]
    teams_df2['gdiffat15'][i] = teams_df2['goldat15'][i]-teams_df2['goldat15'][i-1]
teams_df2.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/loca

,index,gameid,league,ban1,ban2,ban3,ban4,ban5,result,kpm,d,a,fb,fbvictim,fbtime,fd,fdtime,teamdragkills,elementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,fbaron,fbarontime,teambaronkills,dmgtochamps,wards,wardshare,wardkills,wcpm,visionwards,visiblewardclearrate,invisiblewardclearrate,goldspent,minionkills,monsterkillsenemyjungle,goldat10,goldat15,xpat10,gdiffat10,gdiffat15
0,0,1002300127,NALCS,LeBlanc,Zac,Shen,Tristana,Kog'Maw,0,0.118188,18.0,2.0,1,0,6.73213,0,11.941800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.088500,1.0,12.368267,0.0,0,4.0,0.0,21.536467,0.0,32359.0,76.0,1.0,18.0,0.709127,24,0.571429,0.037037,39530.0,691.0,5.0,14949.0,23007.0,18625.0,-342,-3854
1,1,1002300127,NALCS,Caitlyn,Kalista,Thresh,Blitzcrank,Bard,1,0.709127,3.0,55.0,0,1,6.73213,1,11.941800,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,14.088500,0.0,12.368267,1.0,1,10.0,1.0,21.536467,1.0,54497.0,82.0,1.0,27.0,1.063690,25,0.653846,0.2,45160.0,693.0,18.0,15291.0,26861.0,18807.0,342,3854
2,18,240145,NACS,Elise,Caitlyn,Thresh,Vladimir,Brand,1,0.554734,28.0,64.0,1,0,6.53215,0,16.393367,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.826250,0.0,14.792250,1.0,0,10.0,0.0,23.230017,2.0,142669.0,134.0,1.0,36.0,0.798817,23,0.538462,0.147059,80035.0,1117.0,82.0,14972.0,23524.0,17739.0,-79,-1283
3,19,240145,NACS,Zac,Cassiopeia,Nunu,Tristana,Viktor,0,0.621302,25.0,72.0,0,1,6.53215,1,16.393367,3.0,3.0,1.0,0.0,0.0,2.0,0.0,1.0,13.826250,1.0,14.792250,0.0,1,6.0,1.0,23.230017,1.0,139501.0,141.0,1.0,52.0,1.153846,33,0.658537,0.268817,77850.0,1080.0,82.0,15051.0,24807.0,18289.0,79,1283
4,24,240148,NACS,Cassiopeia,Sejuani,Blitzcrank,Lucian,Viktor,1,0.784314,2.0,35.0,1,0,5.72888,1,12.597450,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,16.359083,1.0,7.878567,1.0,1,10.0,1.0,20.965183,1.0,50721.0,68.0,1.0,14.0,0.610022,11,0.733333,0.0714286,42430.0,665.0,33.0,17010.0,27395.0,19356.0,2154,4830


In [0]:
# subset of teams_df columns based on EDA results and calculator goals
cols_teams2 = ["kpm","fbaron","teamdragkills","herald","ft","wards","dmgtochamps","xpat10","gdiffat10","gdiffat15","result"]
teams_df3 = teams_df2[cols_teams2]
teams_df3.head()

,kpm,fbaron,teamdragkills,herald,ft,wards,dmgtochamps,xpat10,gdiffat10,gdiffat15,result
0,0.118188,0.0,0.0,1.0,1.0,76.0,32359.0,18625.0,-342,-3854,0
1,0.709127,1.0,2.0,0.0,0.0,82.0,54497.0,18807.0,342,3854,1
2,0.554734,0.0,1.0,0.0,0.0,134.0,142669.0,17739.0,-79,-1283,1
3,0.621302,1.0,3.0,1.0,1.0,141.0,139501.0,18289.0,79,1283,0
4,0.784314,1.0,2.0,1.0,1.0,68.0,50721.0,19356.0,2154,4830,1


### Save Output

#### Save cleaned files for use at EDA and model development

In [0]:
#drop the extra index column before saving it
teams_df2 = teams_df2.drop(['index'],axis=1)
teams_df2.head() 

,gameid,league,ban1,ban2,ban3,ban4,ban5,result,kpm,d,a,fb,fbvictim,fbtime,fd,fdtime,teamdragkills,elementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,fbaron,fbarontime,teambaronkills,dmgtochamps,wards,wardshare,wardkills,wcpm,visionwards,visiblewardclearrate,invisiblewardclearrate,goldspent,minionkills,monsterkillsenemyjungle,goldat10,goldat15,xpat10,gdiffat10,gdiffat15
0,1002300127,NALCS,LeBlanc,Zac,Shen,Tristana,Kog'Maw,0,0.118188,18.0,2.0,1,0,6.73213,0,11.941800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.088500,1.0,12.368267,0.0,0,4.0,0.0,21.536467,0.0,32359.0,76.0,1.0,18.0,0.709127,24,0.571429,0.037037,39530.0,691.0,5.0,14949.0,23007.0,18625.0,-342,-3854
1,1002300127,NALCS,Caitlyn,Kalista,Thresh,Blitzcrank,Bard,1,0.709127,3.0,55.0,0,1,6.73213,1,11.941800,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,14.088500,0.0,12.368267,1.0,1,10.0,1.0,21.536467,1.0,54497.0,82.0,1.0,27.0,1.063690,25,0.653846,0.2,45160.0,693.0,18.0,15291.0,26861.0,18807.0,342,3854
2,240145,NACS,Elise,Caitlyn,Thresh,Vladimir,Brand,1,0.554734,28.0,64.0,1,0,6.53215,0,16.393367,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.826250,0.0,14.792250,1.0,0,10.0,0.0,23.230017,2.0,142669.0,134.0,1.0,36.0,0.798817,23,0.538462,0.147059,80035.0,1117.0,82.0,14972.0,23524.0,17739.0,-79,-1283
3,240145,NACS,Zac,Cassiopeia,Nunu,Tristana,Viktor,0,0.621302,25.0,72.0,0,1,6.53215,1,16.393367,3.0,3.0,1.0,0.0,0.0,2.0,0.0,1.0,13.826250,1.0,14.792250,0.0,1,6.0,1.0,23.230017,1.0,139501.0,141.0,1.0,52.0,1.153846,33,0.658537,0.268817,77850.0,1080.0,82.0,15051.0,24807.0,18289.0,79,1283
4,240148,NACS,Cassiopeia,Sejuani,Blitzcrank,Lucian,Viktor,1,0.784314,2.0,35.0,1,0,5.72888,1,12.597450,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,16.359083,1.0,7.878567,1.0,1,10.0,1.0,20.965183,1.0,50721.0,68.0,1.0,14.0,0.610022,11,0.733333,0.0714286,42430.0,665.0,33.0,17010.0,27395.0,19356.0,2154,4830


In [0]:
from google.colab import files
#Saving into csvs
teams_df2.to_csv('Teams_v3.csv',index=False)
files.download("Teams_v3.csv") #CSV with features that can still be changed if needed 
teams_df3.to_csv('Teams_v4.csv',index=False)
files.download("Teams_v4.csv") #CSV with final features for calculator

## Players Data

### Handling missing values

In [0]:
cal_missing_val(players_df)

,MissingValue
gameid,0.0
league,0.0
playerid,0.0
position,0.0
champion,0.0
result,0.0
k,0.0
d,0.0
a,0.0
doubles,0.0


In [0]:
# Drop rows with any NaN in the selected column
players_df = players_df.dropna(how = 'any', subset=['gameid'])

# Drop rows with any NaN in the selected columns
players_df = players_df.dropna(how='all', subset=['k','d','a'])

# Drop rows of columns with the same amount of percentages of the missing values
players_df = players_df.dropna(how='all', subset=['fbassist', 'dmgtochamps', 'wards', 'wardkills', 'visionwards', 'visionwardbuys',
                                                  'goldspent', 'minionkills', 'monsterkills', 'monsterkillsownjungle', 'goldat10',
                                                  'goldat15', 'xpat10'])

# Drop rows of columns with any NaN in the selected columns
players_df = players_df.dropna(how='any', subset = ['fbassist', 'dmgtochamps', 'wards', 'wardkills', 'visionwards', 'visionwardbuys',
                                                  'goldspent', 'minionkills', 'monsterkills', 'monsterkillsownjungle', 'goldat10',
                                                  'goldat15', 'xpat10'])


In [0]:
# count the dropped df groupby the gameid and league to make sure that after dropping the data still keep integrity -- which is grouped as 10
# drop the rows where the count is not equal to 10 players (because it's not considered a complete game)
players_gbcount = players_df.groupby(['gameid', 'league']).count()
players_gbcount[players_gbcount['position'] < 10]

,,playerid,position,champion,result,k,d,a,doubles,triples,quadras,pentas,fb,fbassist,fbtime,dmgtochamps,wards,wardkills,visionwards,visionwardbuys,goldspent,minionkills,monsterkills,monsterkillsownjungle,goldat10,goldat15,xpat10
gameid,league,,,,,,,,,,,,,,,,,,,,,,,,,,
1700586,LMS,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9


In [0]:
# drop the groupby results above
players_df = players_df[players_df['gameid'] != 1700586]
players_df.shape

(60570, 28)

### Save Output

In [0]:
from google.colab import files
#Saving into csvs
players_df.to_csv('Players.csv',index=False)
files.download("Players.csv") #csv used for players analysis later